# Strukturální fondy
Loni jsem [se hrabal](https://gist.github.com/kokes/052cc2a63e6366fc92b1) v datech od MMR, konkrétně v tabulce projektů za období 2007-2013. A jelikož to [nikoho nezajímalo](https://twitter.com/kondrej/status/604406720471961600), rozhodl jsem se to udělat znovu a pořádněji.

Vše níže uvedné je na základě [dat na webu MMR](http://www.strukturalni-fondy.cz/cs/Informace-o-cerpani/Seznamy-prijemcu). Rozdíl od loňských hrátek je v tom, že MMR v mezičase dodalo mnohem čistší tabulku. Dříve člověk musel párovat projekty na vládní instituce, NACE kategorie atd. Teď to za nás udělalo ministerstvo, takže můžeme stratifikaci prohlížet mnohem snadněji.

Stížnosti do [twitterů](http://twitter.com/kondrej) nebo [mailů](mailto:ondrej.kokes@gmail.com)

*(Kód používá víceméně jen Python 3 a knihovnu `pandas`, klidně si to pusťte)*

In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
%matplotlib inline

In [2]:
url = 'http://www.strukturalni-fondy.cz/getmedia/d10fe3b0-15f5-4cdb-b916-64be5b4c5c4d/Prehled_projektu_20160406u.xlsx?ext=.xlsx'
fn = 'vstupy/prehled.xlsx'
dvs = 'vystupy' # kam se budou ukladat vystupy

if not os.path.exists(fn):
    urllib.request.urlretrieve(url, fn)

In [3]:
df = pd.read_excel(fn)
df = df.loc[df.loc[:, 'Pořadí v rámci v projektu (filtr)'] == 1] # kazdy projekt ma vic radku

Celkem máme 75 tisíc projektů z období 2007-2013, ke každému 28 indikátorů, většina je celkem sebevysvětlující.

In [4]:
df.shape

(75137, 28)

In [5]:
df.columns

Index(['Číslo a název programu', 'Číslo prioritní osy',
       'Číslo oblasti podpory', 'Číslo projektu', 'Název projektu',
       'Popis projektu', 'Žadatel', 'IČ žadatele', 'Hosp.právní forma',
       'Hosp.právní forma - skupina', 'Stav projektu',
       'Datum podpisu smlouvy/rozhodnutí', 'Adresa žadatele',
       'Kraj žadatele - kód', 'Kraj žadatele - název', 'Obec žadatele - kód',
       'Obec žadatele - název', 'Celkové zdroje - Sml/Dod',
       'Rozhodnutí/Smlouva o poskytnutí dotace (celkové zdroje)',
       'Rozhodnutí/Smlouva o poskytnutí dotace (veřejné prostředky celkem)',
       'Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)',
       'Proplacené prostředky příjemcům - vyúčtované (veřejné prostředky celkem)',
       'Proplacené prostředky příjemcům - vyúčtované (EU zdroje)',
       'Certifikované prostředky (veřejné prostředky celkem)',
       'Certifikované prostředky (EU zdroje)', 'Název NUTS',
       'Pořadí v rámci v projektu (filtr)', 'Počet míst realizace'],
  

In [6]:
# nazvy sloupcu, at se to lip filtruje
prfo = 'Hosp.právní forma'
prfos = 'Hosp.právní forma - skupina'
prog = 'Číslo a název programu'
nuts = 'Název NUTS'
ico = 'IČ žadatele'
zad = 'Žadatel'
adr = 'Adresa žadatele'
obko = 'Obec žadatele - kód'
stav = 'Stav projektu'
popis = 'Popis projektu'
nazev = 'Název projektu'
celk = 'Celkové zdroje - Sml/Dod'
euz = 'Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)'

***V datasetu od MMR je několik metrik pro částku hrazenou z dotací. Jde o Rozhodnutí, Proplacené prostředky a Certifikované prostředky. Je tak možné měřit objem dotací více způsoby. V celém tomto dokumentu se měří pouze ono Rozhodnutí. Změnou proměnné `euz` výše tuto funkcionalitu změníte.***

První tři projekty vypadají takto

In [7]:
df.head(3).T

,0,10,26
Číslo a název programu,CZ.1.01 OP Doprava,CZ.1.01 OP Doprava,CZ.1.01 OP Doprava
Číslo prioritní osy,1.1,1.1,1.1
Číslo oblasti podpory,1.1.1,1.1.1,1.1.1
Číslo projektu,CZ.1.01/1.1.00/06.0019,CZ.1.01/1.1.00/07.0006,CZ.1.01/1.1.00/07.0015
Název projektu,Optimalizace trati Benešov u Prahy - Strančice,Optimalizace trati Horní Dvořiště st. hranice ...,"Rekonstrukce kolejí č.1 a 2 v km 224,391-226,0..."
Popis projektu,"Délka rekonstruovaného úseku je 23,95 km. Stáv...",Vzhledem k již proběhlé částečné optimalizaci ...,V rámci stavby se předpokládá řešit rekonstruk...
Žadatel,"Správa železniční dopravní cesty, státní organ...","Správa železniční dopravní cesty, státní organ...","Správa železniční dopravní cesty, státní organ..."
IČ žadatele,70994234,70994234,70994234
Hosp.právní forma,Správa žel.dopr.cesty,Správa žel.dopr.cesty,Správa žel.dopr.cesty
Hosp.právní forma - skupina,Stát a jeho instituce a organizace,Stát a jeho instituce a organizace,Stát a jeho instituce a organizace


### Další datasety na napojení - veřejné podniky
Sice náš MMR dataset obsahuje informace o státních institucích, seznam není kompletní. Podniky s výrazným podílem státu nebo firmy plně vlastněné kraji, obcemi atd., jsou v datasetu často označené jako běžné akciové společnosti.

Můžeme ale použít dva datasety of ministerstva financí:
- [Seznam vládních institucí v ČR](http://www.mfcr.cz/cs/verejny-sektor/rozpoctove-ramce-statisticke-informace/verejny-sektor/sektor-vladnich-instituci/2016/seznam-vladnich-instituci-v-cr-2016-24749)
- [Seznam veřejných společností v ČR](http://www.mfcr.cz/cs/verejny-sektor/rozpoctove-ramce-statisticke-informace/verejny-sektor/verejne-spolecnosti/2016/seznam-verejnych-spolecnosti-v-cr-2016-24752)

In [8]:
url = 'http://www.mfcr.cz/assets/cs/media/Rozp-ramce-EU-85-2011_2016_Seznam-verejnych-spolecnosti-v-CR.xlsx'
fn = 'vstupy/verejne-instituce.xlsx'

if not os.path.exists(fn):
    urllib.request.urlretrieve(url, fn)
    
url = 'http://www.mfcr.cz/assets/cs/media/Rozp-ramce-EU-85-2011_2016_Seznam-vladnich-instituci-v-CR.xls'
fn = 'vstupy/vladni-instituce.xls'

if not os.path.exists(fn):
    urllib.request.urlretrieve(url, fn)

In [9]:
ver = pd.read_excel('vstupy/verejne-instituce.xlsx', skiprows=4)
ver = ver.iloc[:, 1:]
ver.head(5)

,IČO,NÁZEV,DATUM VZNIKU,FORMA,NACE,SUBSEKTOR
0,46967036,"JUPITER club, s.r.o.",1992-08-03,112,90040,11001
1,48909467,"Služby města Konice, spol. s r.o.",1993-08-18,112,38110,11001
2,25013599,TS HRÁDEK s.r.o.,1996-12-19,112,38110,11001
3,25013866,"Klášterecká kyselka, s.r.o.",1996-12-31,112,47190,11001
4,25020307,Podnik služeb Jirkov s.r.o.,1997-05-06,112,81290,11001


In [10]:
vlad = pd.read_excel('vstupy/vladni-instituce.xls', skiprows=4)
vlad = vlad.iloc[:, 1:]
vlad.head(5)

,IČO,NÁZEV,DATUM VZNIKU,FORMA,NACE,COFOG,SUBSEKTOR
0,26925974,Nemocnice Boskovice s.r.o,2004-04-22,112,86100,NaN,13130
1,3960765,Lesy Drmoul s.r.o.,2015-04-03,112,2100,NaN,13130
2,27577708,Divadla Kladno s.r.o.,2006-08-02,112,90020,NaN,13130
3,26076357,"Strakonická televize, s.r.o.",2004-07-10,112,60200,NaN,13130
4,4510984,"Pošumavská odpadová, s.r.o.",2015-10-26,112,38,NaN,13130


In [11]:
len(ver), len(vlad)

(1172, 18117)

In [12]:
vvico = list(ver.loc[:, 'IČO'].values) + list(vlad.loc[:, 'IČO'].values)
vvico = [str(l).rjust(8, '0') for l in vvico]
vvico = set(vvico)

## První prohrab

Hlavní přídavek oproti předchozímu datasetu je přítomnost klasifikací dle pravních forem.

In [13]:
df.loc[:, prfo].unique()[:10]

array(['Správa žel.dopr.cesty',
       'Správa železniční dopravní cesty, státní organizace',
       'Společnost s r.o.', 'Akciová společnost', 'Příspěvková organizace',
       'Státní příspěvková organizace', 'Kraj', 'Dobrovolný svazek obcí',
       'Obec nebo m.část hl.m.', 'Org. složka státu'], dtype=object)

Ty jsou seskupené do skupin. Nejvíc projektů (ne nutně peněz) připadá na obce.

In [14]:
df.loc[:, prfos].value_counts()

Obce                                   25192
Podnikatelské subjekty                 22118
Ostatní                                15068
Komory, profesní a zájmové sdružení     3221
Stát a jeho instituce a organizace      2542
Neziskový sektor                        1601
Kraje                                   1584
Vzdělávací a výzkumné instituce         1550
Name: Hosp.právní forma - skupina, dtype: int64

## Očista

### Nemáme žadatele ani IČO

In [15]:
msk = df.loc[:, zad].isnull() & df.loc[:, ico].isnull()
df.loc[msk].head(3).iloc[:, :8]

,Číslo a název programu,Číslo prioritní osy,Číslo oblasti podpory,Číslo projektu,Název projektu,Popis projektu,Žadatel,IČ žadatele
1487,CZ.1.01 OP Doprava,1.6,1.6.3,CZ.1.01/6.3.00/14.0392,Výměna dnové obšívky plavidla MS BEATRICE,Realizace projektu je nutná z důvodu havarijní...,NaN,NaN
3007,CZ.1.02 OP Životní prostředí,2.2,2.2.1,CZ.1.02/2.1.00/09.03845,Rekonstrukce vytápění hvozdu sladovny Kounice,Projekt rekonstrukce vytápění hvozdu sladovny ...,NaN,NaN
3058,CZ.1.02 OP Životní prostředí,2.2,2.2.1,CZ.1.02/2.1.00/11.09845,Zateplení a výměna zdroje vytápění,Výměna starého neekologického zdroje vytápění ...,NaN,NaN


Většinou živnostníci.

In [16]:
df.loc[msk,euz].sum()/10**9

8.2611358485200004

Dohromady 8 miliard.

*Nastavíme nějaké prázdné hodnoty*

In [17]:
df.loc[msk, zad] = 'Neidentifikován'
df.loc[msk, ico] = '9'*8

### Nesprávné IČO
Je spousta projektů, kde IČO není správné české IČO. U zhruba 150 případů IČO chybí, u cca 1800 jde o zahraniční IČO (většina projektů ze spolupráce Česko-Polsko).

In [18]:
df.loc[:, ico].isnull().sum()

1740

In [19]:
nims = df.loc[:, ico].apply(lambda x: len(str(x)) == 8)
len(df)-nims.sum()

3408

**Takže smažeme všechny projekty, kde IČO nemá osm znaků, ale zároveň není prázdné (na ty se ještě podíváme).**

In [20]:
df = df.loc[~((~df.loc[:, ico].isnull()) & ~nims)]

#### Dvě stejně jmenné firmy, jedna má IČO, jedna nemá

In [21]:
ddd=df.loc[:, [zad, ico, obko]]
dg = ddd.groupby(zad)
for gr in dg:
    uq = gr[1].loc[:, ico].unique()
    if len(uq) == 1: continue
    #if gr[0].lower().startswith('obec'): continue
    #if gr[0].lower().startswith('město'): continue
    if len(gr[1].loc[:, obko].unique()) > 1: continue # jsou z ruznych obci, preskocit
    
    if len(uq) == 2 and pd.Series(uq).isnull().sum() == 1:
        # print(gr[0], uq)
        ic = gr[1].loc[:, ico].dropna().unique()[0] # ICO tyhle firmy
        df.loc[df.loc[:, zad] == gr[0], ico] = ic

#### Co zbývá

In [22]:
df.loc[:, ico].isnull().sum()

273

### Ruční IČO

In [23]:
nl = df.loc[:, ico].isnull()
df.loc[nl, [zad, ico, nuts]].drop_duplicates().head(10)

,Žadatel,IČ žadatele,Název NUTS
21377,"Českomoravská záruční a rozvojová banka, a.s.",NaN,Praha 1
27914,Agentura pro podporu podnikání a investic Czec...,NaN,Praha 2
34907,Česká agentura na podporu obchodu/CzechTrade,NaN,Praha 2
42486,Zaměstnavatelský svaz důlního a naftového prům...,NaN,Kladno
56143,OBEC KOSTELEC U HOLEŠOVA,NaN,Kostelec u Holešova
56144,OBEC CHOMÝŽ,NaN,Chomýž
60817,OBEC RŮŽOVÁ,NaN,Růžová
60818,OBEC VELKÁ BUKOVINA,NaN,Velká Bukovina
60919,Obec Sobíšky,NaN,Sobíšky
60936,OBEC HRACHOLUSKY,NaN,Hracholusky


Ručně jsem dohledal asi 150 IČO kódů

In [24]:
ruc = pd.read_csv('vstupy/rucni-ico.csv', dtype=object)
ruc.head(3)

,Žadatel,IČ žadatele,Název NUTS
0,"Českomoravská záruční a rozvojová banka, a.s.",44848943,Praha 1
1,Česká agentura na podporu obchodu/CzechTrade,00001171,Praha 2
2,OBEC KOSTELEC U HOLEŠOVA,00287342,Kostelec u Holešova


Tak je uložíme zpět

In [25]:
for el in ruc.iterrows():
    zz = el[1].loc['Žadatel']
    ii = el[1].loc['IČ žadatele']
    df.loc[(df.loc[:, zad] == zz) & (df.loc[:, ico].isnull()), ico] = ii

Zbylo toho fakt málo

In [26]:
df.loc[df.loc[:, ico].isnull(), zad].unique()

array(['Stowarzyszenie Gmin Polskich Euroregionu Glacensis',
       'Stowarzyszenie Gmin Dorzecza Górnej Odry',
       'Stowarzyszenie "Region Beskidy"',
       'Stowarzyszenie Gmin Polskich Euroregionu Pradziad'], dtype=object)

In [27]:
df = df.loc[~df.loc[:, ico].isnull()]

O tyto projekty náš dataset očistíme.

### Duplikované skupiny

In [28]:
df.loc[:, prfo] = df.loc[:, prfo].replace('Společen.vlast.jednot.', 'Společenství vlastníků jednotek')
df.loc[:, prfo] = df.loc[:, prfo].replace('Reg.rada regionu soudr.', 'Regionální rada regionu soudržnosti')

In [29]:
df.loc[:, prfo] = df.loc[:, prfo].replace('Vysoká škola (veřejná, státní)', 'Vysoká škola')

In [30]:
df.loc[:, prfo] = df.loc[:, prfo].replace('Příspěvková organizace', 'Státní příspěvková organizace')

In [31]:
df.loc[:, prfo] = df.loc[:, prfo].replace('Veř. výzkumná instituce', 'Veřejná výzkumná instituce')

In [32]:
df.loc[:, prfo] = df.loc[:, prfo].replace('Obecně prospěšná spol.', 'Obecně prospěšná společnost')

### Veřejné instituce
Čtyři skupiny institucí patří mezi veřejné - obce, kraje, stát, vzdělávací a výzkumné instituce.

In [33]:
erar = 'veřejná instituce'
df.loc[:, erar] = False
df.loc[df.loc[:, prfos].isin(['Obce', 'Kraje', 'Stát a jeho instituce a organizace', 'Vzdělávací a výzkumné instituce']), erar] = True

...ale v 'Ostatní' je i několik skupin, které jsou veřejné.

In [34]:
list(df.loc[df.loc[:, prfos] == 'Ostatní', prfo].unique())

['Státní příspěvková organizace',
 'Příspěvková organizace zřízená územním samosprávným celkem',
 'Fyzická osoba nepodnikající dle Živnost. zákona a nezapsaná v OR',
 'Ústav',
 'Odštěp.záv.zaps.v ob.r.',
 'Společenství vlastníků jednotek',
 'Regionální rada regionu soudržnosti',
 'Zatím neurčeno']

In [35]:
df.loc[df.loc[:, prfo].isin(['Státní příspěvková organizace',
 'Příspěvková organizace zřízená územním samosprávným celkem',
 'Regionální rada regionu soudržnosti']), erar] = True

Taky je pár veřejných institucí mezi žadateli, kteří nemají hlavní nadskupinu.

In [36]:
df.loc[df.loc[:, prfos].isnull(),prfo].unique()[:6]

array(['Veřejná výzkumná instituce', 'Školská právnická osoba',
       'Evropská společnost',
       'Odborová organizace a organizace zaměstnavatelů', 'Ostatní',
       'Zem.podnik.FO nez.v OR'], dtype=object)

In [37]:
df.loc[df.loc[:,prfo].isin(['Veřejná výzkumná instituce', 'Školská právnická osoba']), erar] = True

Poslední zdroj informací budou ty dva datasety of MFČR, které jsme načetli výše. Projdeme tedy podniky, které jsou označené jako soukromé a podíváme se, jestli jeji IČO není v těch dvou datasetech.

In [38]:
dd = df.loc[df.loc[:,erar] == False, [ico, erar]]
for j in dd.iterrows():
    if j[1].loc[ico] not in vvico: continue
    df.loc[j[0], erar] = True

### EIB
Poslední výjimka, v datech je pod IČO 00000000 Evropská investiční banka, EIB. Tak ji přepneme do veřejných.

In [39]:
df.loc[df.loc[:, ico] == '00000000']

,Číslo a název programu,Číslo prioritní osy,Číslo oblasti podpory,Číslo projektu,Název projektu,Popis projektu,Žadatel,IČ žadatele,Hosp.právní forma,Hosp.právní forma - skupina,...,Rozhodnutí/Smlouva o poskytnutí dotace (veřejné prostředky celkem),Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje),Proplacené prostředky příjemcům - vyúčtované (veřejné prostředky celkem),Proplacené prostředky příjemcům - vyúčtované (EU zdroje),Certifikované prostředky (veřejné prostředky celkem),Certifikované prostředky (EU zdroje),Název NUTS,Pořadí v rámci v projektu (filtr),Počet míst realizace,veřejná instituce
100818,CZ.1.10 ROP NUTS II Moravskoslezsko,10.2,10.2.3,CZ.1.10/2.3.00/02.00976,JESSICA Holdingový fond Moravskoslezsko,Evidence financování JESSICA Holdingového fond...,Evropská investiční banka,00000000,Zahraniční osoba,Podnikatelské subjekty,...,430882000.0,506920000.0,430882000.0,506920000.0,430882000.0,CZ0802598003,Frýdek-Místek,1.0,5.0,False


In [40]:
df.loc[df.loc[:, zad] == 'Evropská investiční banka', erar] = True

### Kofinancování

In [41]:
kofi = 'poměr vlastních financí (0-1)'
df.loc[:, kofi] = np.nan
fr = 1-df.loc[:, euz] / df.loc[:, celk]

fr = fr[fr >= 0] # mám pocit že u těch českopolských projektů je celková cena v eurech, ale dotace v korunách (a zlotých)
df.loc[fr.index, kofi] = fr.values

### Negativní dotace

**Negativní dotace.** Nezkoumal sem, jestli chyba nebo pokuta nebo co.

In [42]:
df.loc[df.loc[:, euz]<0, [zad, euz]]

,Žadatel,Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)
79295,Odborná střední škola podnikatelská Kolín s.r.o.,-5.820766e-11
112983,AAA BYTY.CZ akciová společnost,-2.849530e+05


### Duplikovaní žadatelé
Máme víc žadatelů než IČO.

In [43]:
print(len(df.loc[:, zad].unique()))


print(len(df.loc[:, ico].unique()))

26390
24355


In [44]:
dd = df.loc[:, [zad,ico]].drop_duplicates() #.sort_values(by=ico)
dd.index = range(len(dd))
dd.head(3)

,Žadatel,IČ žadatele
0,"Správa železniční dopravní cesty, státní organ...",70994234
1,GJW Praha spol. s r.o.,41192869
2,"České dráhy, a.s.",70994226


In [45]:
df.loc[:, [zad, ico]].drop_duplicates().shape

(27028, 2)

In [46]:
dd = df.loc[:, [zad, ico]].drop_duplicates()
dd = dd.loc[dd.duplicated(subset=ico, keep=False)].sort_values(by=ico)
dd.head(10)

,Žadatel,IČ žadatele
54103,Česká agentura na podporu obchodu,00001171
34907,Česká agentura na podporu obchodu/CzechTrade,00001171
54269,Ministerstvo financí,00006947
54040,MINISTERSTVO FINANCÍ,00006947
6075,ČESKÁ SPRÁVA SOCIÁLNÍHO ZABEZPEČENÍ,00006963
10340,Česká správa sociálního zabezpečení,00006963
7752,Ministerstvo vnitra,00007064
50573,MINISTERSTVO VNITRA,00007064
54236,Státní veterinární správa,00018562
54096,STÁTNÍ VETERINÁRNÍ SPRÁVA ČR,00018562


Můžem nahradit duplikovaný názvy.

In [47]:
for j in dd.drop_duplicates(subset=ico).iterrows():
    df.loc[df.loc[:, ico] == j[1].loc[ico], zad] = j[1].loc[zad]

**Furt nějaký zbývaj**

In [48]:
print(len(df.loc[:, zad].unique()))

print(len(df.loc[:, ico].unique()))

23865
24355


Hotovo

**Bude snadný vyřešit líp až doděláme data z RES**

## Operační programy

In [49]:
prdf = df.loc[:, [prog, euz, erar]].groupby([prog, erar]).sum()
prdf /= 10**6
prdf = prdf.round(1).sort_values(by=euz, ascending=False).unstack()
prdf

Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)  \
veřejná instituce                                                                          False   
Číslo a název programu                                                                             
CZ.1.01 OP Doprava                                                                         200.8   
CZ.1.02 OP Životní prostředí                                                             24820.4   
CZ.1.03 OP Podnikání a inovace                                                           86612.1   
CZ.1.04 OP Lidské zdroje a zaměstnanost                                                  15505.8   
CZ.1.05 OP Výzkum a vývoj pro inovace                                                     4631.9   
CZ.1.06 Integrovaný operační program                                                      5516.9   
CZ.1.07 OP Vzdělávání pro konkurenceschopnost                                             9331.7   
CZ.1.08 OP Technická pomoc                                                                 106.5   
CZ.1.09 ROP NUTS II Severozápad                                                           1934.1   
CZ.1.10 ROP NUTS II Moravskoslezsko                                                       1924.2   
CZ.1.11 ROP NUTS II Jihovýchod                                                            2171.1   
CZ.1.12 ROP NUTS II Střední Morava                                                        2828.8   
CZ.1.13 ROP NUTS II Severovýchod                                                          2940.3   
CZ.1.14 ROP Jihozápad                                                                     2136.0   
CZ.1.15 ROP NUTS II Střední Čechy                                                         1318.3   
CZ.1.25 OP Rybářství                                                                       693.5   
CZ.2.16 OP Praha Konkurenceschopnost                                                      1009.6   
CZ.2.17 OP Praha Adaptibilita                                                             1840.8   
CZ.3.22 OP ČR - Polsko                                                                      93.7   

                                                         
veřejná instituce                                 True   
Číslo a název programu                                   
CZ.1.01 OP Doprava                             191090.5  
CZ.1.02 OP Životní prostředí                   119224.1  
CZ.1.03 OP Podnikání a inovace                  12563.9  
CZ.1.04 OP Lidské zdroje a zaměstnanost         40853.6  
CZ.1.05 OP Výzkum a vývoj pro inovace           50119.7  
CZ.1.06 Integrovaný operační program            39112.7  
CZ.1.07 OP Vzdělávání pro konkurenceschopnost   42229.6  
CZ.1.08 OP Technická pomoc                       3980.3  
CZ.1.09 ROP NUTS II Severozápad                 19759.8  
CZ.1.10 ROP NUTS II Moravskoslezsko             20866.1  
CZ.1.11 ROP NUTS II Jihovýchod                  20891.7  
CZ.1.12 ROP NUTS II Střední Morava              18645.5  
CZ.1.13 ROP NUTS II Severovýchod                18827.8  
CZ.1.14 ROP Jihozápad                           18431.9  
CZ.1.15 ROP NUTS II Střední Čechy               17304.8  
CZ.1.25 OP Rybářství                              207.2  
CZ.2.16 OP Praha Konkurenceschopnost             6262.2  
CZ.2.17 OP Praha Adaptibilita                    1292.0  
CZ.3.22 OP ČR - Polsko                            519.0

In [50]:
prdf.to_csv(os.path.join(dvs, 'programy.csv'))

In [51]:
(df.loc[:, [celk, euz, erar]].groupby(erar).sum()/10**9).round(0)

,Celkové zdroje - Sml/Dod,Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)
veřejná instituce,,
False,339.0,166.0
True,864.0,642.0


Jde tedy o cca 800 mld na projekty za 1.2 bilionu, kofinance jsou tedy zhruba třetinová.

In [52]:
df.loc[:, [kofi,erar]].groupby(erar).describe().unstack()

poměr vlastních financí (0-1)                           \
                                          count      mean       std  min   
veřejná instituce                                                          
False                                   27056.0  0.397615  0.248035  0.0   
True                                    44361.0  0.163042  0.183346  0.0   

                                                           
                        25%       50%       75%       max  
veřejná instituce                                          
False              0.141643  0.498908  0.600000  1.358232  
True               0.002775  0.105371  0.256298  1.000000

**chyba v mejch počtech (max > 1)**

## Nejvíc

### Veřejné

In [53]:
top = df.groupby([zad,ico]).sum().loc[:, [euz]].sort_values(by=euz, ascending=False)/10**9
top.round(1).head(20)

,,Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)
Žadatel,IČ žadatele,
Ředitelství silnic a dálnic ČR,65993390,96.1
"Správa železniční dopravní cesty, státní organizace",70994234,79.5
MINISTERSTVO PRÁCE A SOCIÁLNÍCH VĚCÍ,00551023,20.2
"Dopravní podnik hl.m. Prahy ,akciová společnost",00005886,11.9
Moravskoslezský kraj,70890692,10.9
Masarykova univerzita,00216224,10.8
Neidentifikován,99999999,8.3
Ministerstvo vnitra,00007064,7.8
"Úřad práce ČR - Krajská pobočka v Brně, kontaktní pracoviště Blansko",72496991,7.7


In [54]:
top.head(1000).to_csv(os.path.join(dvs, 'top-verejne.csv'))

### Privátní

In [55]:
top_priv = df.loc[~df.loc[:, erar], :]
top_priv = top_priv.groupby([zad,ico]).sum().loc[:, [euz]].sort_values(by=euz, ascending=False)/10**9
top_priv.head(20)

,,Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)
Žadatel,IČ žadatele,
Neidentifikován,99999999,8.261136
"TŘINECKÉ ŽELEZÁRNY, a. s.",18050646,2.300128
ArcelorMittal Ostrava a.s.,45193258,1.676587
Dolní oblast VÍTKOVICE,75125285,1.273931
"Veolia Energie ČR, a.s.",45193410,0.956849
"VÚTS, a.s.",46709002,0.770943
Svaz VKMO s.r.o.,25572245,0.703694
Svaz měst a obcí České republiky,63113074,0.657223
"Unipetrol výzkumně vzdělávací centrum, a.s.",62243136,0.587261


In [56]:
top_priv.head(1000).to_csv(os.path.join(dvs, 'top-soukrome.csv'))

## Golf a hotely

**TODO**: dodelat podle NACE

In [77]:
len(df.loc[df.loc[:, nazev].str.contains('golf')])

6

In [78]:
df.loc[df.loc[:, 'Název projektu'].str.contains('golf')].sum()/10**6

Celkové zdroje - Sml/Dod                                                    254.482285
Rozhodnutí/Smlouva o poskytnutí dotace (celkové zdroje)                      99.682815
Rozhodnutí/Smlouva o poskytnutí dotace (veřejné prostředky celkem)           85.169353
Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)                           83.445630
Proplacené prostředky příjemcům - vyúčtované (veřejné prostředky celkem)     70.928785
Proplacené prostředky příjemcům - vyúčtované (EU zdroje)                     83.445630
Certifikované prostředky (veřejné prostředky celkem)                         70.928785
Pořadí v rámci v projektu (filtr)                                             0.000006
Počet míst realizace                                                          0.000008
veřejná instituce                                                             0.000001
poměr vlastních financí (0-1)                                                 0.000003
dtype: float64

## Neziskovky

**Nekompletní, bude až z právní formy z RES, tohle obšancovávání je nepřesný**

In [108]:
df.loc[df.loc[:, prfos] == 'Komory, profesní a zájmové sdružení', prfo].unique()

array(['Zájmové sdružení právnických osob', 'Spolek', 'Pobočný spolek',
       'Zájm.sdružení práv.osob', 'Komora s vyj.prof.komor',
       'Sdruž.(svaz,spolek,spol', 'Org.jednotka sdružení',
       'Stav.org.-profesní kom.', 'Komora (hospodářská, agrární)',
       'Mezinár. org.a sdružení', 'Zájmové sdružení',
       'Organizační jednotka občanského sdružení',
       'Mezinárodní nevládní organizace'], dtype=object)

In [104]:
nzs = df.loc[(df.loc[:, prfos] == 'Neziskový sektor') | (df.loc[:, prfo] == 'Ústav')]
nzs.sum()/10**6

Celkové zdroje - Sml/Dod                                                    12389.733936
Rozhodnutí/Smlouva o poskytnutí dotace (celkové zdroje)                     11048.307980
Rozhodnutí/Smlouva o poskytnutí dotace (veřejné prostředky celkem)           9527.357691
Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)                          10003.198079
Proplacené prostředky příjemcům - vyúčtované (veřejné prostředky celkem)     8625.621203
Proplacené prostředky příjemcům - vyúčtované (EU zdroje)                     6708.710529
Certifikované prostředky (veřejné prostředky celkem)                         5777.701690
Pořadí v rámci v projektu (filtr)                                               0.001609
Počet míst realizace                                                            0.005024
veřejná instituce                                                               0.000032
poměr vlastních financí (0-1)                                                   0.000181
dtype: float64

1600 projektů

In [105]:
nzs.shape

(1609, 30)

In [81]:
nejnzs = nzs.groupby(zad).sum().sort_values(ascending=False, by=euz)/10**6

nejnzs.loc[:, [euz]].head(10).round(0)

,Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)
Žadatel,
Techmania Science Center o.p.s.,572.0
Kanonie premonstrátů Teplá,497.0
Římskokatolická farnost Velehrad,408.0
"IQLANDIA, o.p.s.",407.0
Federace židovských obcí v České republice,285.0
"Energeia, o.p.s.",254.0
"Člověk v tísni, o.p.s.",244.0
Benediktinské opatství sv. Václava v Broumově,196.0
"Muzeum a společenské centrum Dobrovice, o.p.s.",163.0


In [89]:
nejnzs.loc[:, [euz]].head(1000).to_csv(os.path.join(dvs, 'neziskovky.csv'))

neziskovky bez církví

In [91]:
nzsb = nzs.loc[~nzs.loc[:, prfo].str.contains('írk')]

In [92]:
dfn = pd.DataFrame(nzsb.groupby([prfo, zad]).sum().loc[:, euz]/10**6)
dfn.sort_values(by=euz, ascending=False).head(10).round(0)

Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)
Hosp.právní forma           Žadatel                                                                                               
Obecně prospěšná společnost Techmania Science Center o.p.s.                                                                 572.0 
                            IQLANDIA, o.p.s.                                                                                407.0 
                            Energeia, o.p.s.                                                                                254.0 
                            Člověk v tísni, o.p.s.                                                                          244.0 
                            Muzeum a společenské centrum Dobrovice, o.p.s.                                                  163.0 
                            Centrála cestovního ruchu Východní Moravy, o.p.s.                                               136.0 
                            OSTROV ZDRAVÍ o.p.s.                                                                            128.0 
Nadace                      ,, Nadace Josefa Viewegha Sklářská huť František "                                              127.0 
                            Nadace Partnerství (v mezinárodním styku "Czech...                                               92.0 
Obecně prospěšná společnost Vyšší odborná škola DAKOL a Střední škola DAKOL...                                               85.0

In [94]:
dfn.sort_values(by=euz, ascending=False).head(1000).to_csv(os.path.join(dvs, 'neziskovky-bez-cirkvi.csv'))

## Obce
per capita? jsou někde deviace?

Škromachovic

In [85]:
df.loc[:, zad].str.contains('Rohatec').sum()

4

In [86]:
df.loc[df.loc[:, nuts] == 'Rohatec'].sum()/10**6

Celkové zdroje - Sml/Dod                                                    326.411804
Rozhodnutí/Smlouva o poskytnutí dotace (celkové zdroje)                     183.536061
Rozhodnutí/Smlouva o poskytnutí dotace (veřejné prostředky celkem)          163.902748
Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)                          168.626124
Proplacené prostředky příjemcům - vyúčtované (veřejné prostředky celkem)    150.920879
Proplacené prostředky příjemcům - vyúčtované (EU zdroje)                    144.642873
Certifikované prostředky (veřejné prostředky celkem)                        129.313003
Pořadí v rámci v projektu (filtr)                                             0.000037
Počet míst realizace                                                          0.000037
veřejná instituce                                                             0.000011
poměr vlastních financí (0-1)                                                 0.000016
dtype: float64

In [87]:
(df.loc[df.loc[:, nuts] == 'Rohatec', [prog, euz, erar]].groupby([erar, prog]).sum()/10**6).unstack().round(0)

Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)  \
Číslo a název programu                       CZ.1.02 OP Životní prostředí   
veřejná instituce                                                           
False                                                                15.0   
True                                                                 26.0   

                                                       \
Číslo a název programu CZ.1.03 OP Podnikání a inovace   
veřejná instituce                                       
False                                           111.0   
True                                              NaN   

                                                                \
Číslo a název programu CZ.1.04 OP Lidské zdroje a zaměstnanost   
veřejná instituce                                                
False                                                      4.0   
True                                                       NaN   

                                                             \
Číslo a název programu CZ.1.06 Integrovaný operační program   
veřejná instituce                                             
False                                                   NaN   
True                                                    0.0   

                                                                      \
Číslo a název programu CZ.1.07 OP Vzdělávání pro konkurenceschopnost   
veřejná instituce                                                      
False                                                            NaN   
True                                                             3.0   

                                                       
Číslo a název programu CZ.1.11 ROP NUTS II Jihovýchod  
veřejná instituce                                      
False                                             NaN  
True                                              9.0

In [88]:
ver.loc[:, 'NÁZEV'].str.contains('Holovo').sum()

0